In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

### data

In [2]:
X = pd.read_csv('mnist_data.csv').values[:, 1:]
y = pd.read_csv('mnist_target.csv').values[:, 1]
X = X / 255
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [3]:
def mini_batches(X, y, batch_size):
    m, _ = X.shape
    indices = list(range(m))
    X_, y_ = torch.Tensor(X).float(), torch.Tensor(y).long()
    X_, y_ = X_[indices], y_[indices]
    pos = 0
    while pos < m:
        X_batch, y_batch = X_[pos:pos+batch_size], y_[pos:pos+batch_size]
        yield (X_batch, y_batch)
        pos += batch_size

### model

In [4]:
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x), dim=1)

### model training

In [21]:
LR = 5e-3
MOMENTUM = 0.9
input_size = 784
hidden_sizes = [128, 64]
output_size = 10
batch_size = 64

classifier = Classifier(input_size, hidden_sizes, output_size)
optimizer = optim.SGD(classifier.parameters(), lr=LR, momentum=MOMENTUM)
loss_function = nn.NLLLoss()

In [22]:
n_epochs = 30
losses = []

def train():
    for i in range(n_epochs):
        epoch_loss = 0
        for batch in mini_batches(X_train, y_train, 64):
            X_, y_ = batch
            y_hat = classifier(X_)
            optimizer.zero_grad()
            loss = loss_function(y_hat, y_)
            epoch_loss += loss
            loss.backward()
            optimizer.step()
        print(f'{i} epoch loss = {epoch_loss}')
        losses.append(epoch_loss)

In [ ]:
train()

0 epoch loss = 643.4887084960938
1 epoch loss = 235.7663116455078
2 epoch loss = 179.10635375976562
3 epoch loss = 143.241943359375
4 epoch loss = 119.12183380126953
5 epoch loss = 101.10900115966797


In [ ]:
import matplotlib.pyplot as plt

plt.plot(np.arange(1, n_epochs+1), losses)
plt.title('losses')
plt.xlabel('epoch', fontsize=10)
plt.ylabel('loss', fontsize=10)
plt.show()

## adversarial attack

In [ ]:
def fast_adversarial_example(dig, n=784, n_classes=10, eta=0.01, iters=10):
    y = torch.tensor([dig]).long()
    x = Variable(torch.tensor(np.random.normal(loc=0, scale=0.01, size=(1,n))).float(), requires_grad=True)
    for _ in range(iters):
        y_hat = classifier(x)
        classifier.zero_grad()
        loss = loss_function(y_hat, y)
        loss.backward()
        grad = x.grad.data
        x = Variable(torch.clamp(x - eta * grad.sign(), 0, 1), requires_grad=True)
    return x

In [ ]:
example = fast_adversarial_example(8, eta=0.1, iters=5)
alpha = 0.5
X_spoiled = np.maximum(X_test*(1-alpha), alpha*example.detach().numpy())
x_target = X_spoiled[0]

In [ ]:
def predict(x):
    y_hat = classifier(x)
    return torch.argmax(y_hat), y_hat

In [ ]:
y_pred, score = predict(x_target.unsquuese(0))